**Final Project - Adaptive Procedure for Threshold Estimation**
  
  
  
  
Contributors: Enya Walker, Kristie Flores 

Course: COGS/PSYCH 14P   

----------------------------------------------------------

EXPERIMENTER INSTRUCTIONS:

This program tests frequency discrimination thresholds through an adaptive staircase procedure. Multiple two-tone trials will be presented to a subject, who must indicate as accurately as possible which tone has the highest frequency. The frequency difference between tones will adapt based on the correctness of the response. Subjects will be provided five test trials to confirm their comprehension of the instructions. If comprehension is confirmed, the experiment will begin. If not, subjects must review instructions with the experimenter.  

Changeable test parameters are accessible under "SET PARAMETERS." *For each new subject, the subject ID# (subjid) must be updated accordingly.* Variables such as base frequencies to test (f), initial frequency difference (df), maximum trials before experiment termination (max_trials), and maximum error before termination (max_error) may also be changed as needed. For safety, adjust the program's volume (vol) gradually and *do not exceed 0.5.*

After setting experiment parameters, continue to the section titled "BEGIN EXPERIMENT" and run the code. The test and experimental trials will proceed automatically, with instructions provided for the subject. Finally, a .csv file containing subject information and results will be saved under "Experiment_Subject*ID#*.csv."

----------------------------------------------------------

**SET PARAMETERS**

In [3]:
'''IMPORT PACKAGES'''

import numpy as np       
from numpy import random 
import sounddevice as sd
import pandas as pd

'''ADJUST EXPERIMENT PARAMETERS'''

#SUBJECTS
subjid = 6 #ID of subject tested
rand = subjid #Random number generator seed - currently set to subject ID

#STIMULI
f = [440,880,1760] #Base frequencies
df = 0.15  #Starting frequency difference
vol = 0.05 #Volume - for safety, do NOT exceed 0.5

#DESIGN FEATURES
test_trials = 5 #Number of test trials
max_error = 6 #After this many errors, the experiment will terminate
max_trials = 100 #After this many total trials, the experiment will terminate

**BEGIN EXPERIMENT**

In [4]:
'''PRE-SET PARAMETERS'''

rng = random.default_rng(seed=rand)
sr = 44100 #Sampling rate
duration = 0.5 #Duration of tone 

silence_duration = 0.5 #Duration of silence between tones
silence_samples = silence_duration*sr 
silence = np.zeros(int(silence_samples))

f = list(f)

'''FUNCTIONS FOR STIMULUS CONSTRUCTION'''

def make_tone(f,duration= 0.5,sr = 44100,ramp = 300):
    time_vec = np.linspace(0, duration, int(duration*sr)) 
    tone = np.sin(f * time_vec  * 2 * np.pi)
    ramp_up = np.linspace(0,1,ramp)
    ramp_down = np.linspace(1,0,ramp)
    tone[0:ramp] = tone[0:ramp]*ramp_up
    tone[-ramp:] = tone[-ramp:]*ramp_down
    return tone

def play_sound(tone,vol = 0.05,sr = 44100):
    tone  = vol*tone/np.max(np.abs(tone)) 
    tone = 32768*tone 
    tone  = tone.astype(np.int16) 
    sd.play(tone, sr,blocking=True) 

'''INTRODUCTION TO EXPERIMENT'''

intro_check = False
while intro_check == False:
    intro = input("In this study, you will be given a series of prompts. For each, listen to the notes played and indicate to the best of your ability which note has a higher pitch. If the first is highest, type '1'. If the second is highest, type '2.' Before the study begins, you will be provided five test prompts to practice responding. Type 'y' to continue.")
    if (intro == 'y') | (intro == 'Y'):
        intro_check = True
    else:
        intro_check = False

'''TEST TRIALS'''

#INITIALIZE TEST TRIALS
test_order = rng.integers(1,3,test_trials)
testrecord = []
testresponses = []
test_freq = 0 #Index into the base frequency used for testing
f2 = f[test_freq]+(f[test_freq]*df)

#CONDUCT TEST TRIALS
if intro_check == True:
    while test_freq < 1:
        freq_lo = make_tone(f[test_freq],duration)
        freq_hi = make_tone(f2,duration)
        test_2 = np.concatenate((freq_lo,silence,freq_hi)) #Low-high trial; correct answer = 2
        test_1 = np.concatenate((freq_hi,silence,freq_lo)) #High-low trial; correct answer = 1
        for j in range(test_trials):
            if test_order[j] == 1: #High-low trial implemented
                play_sound(test_1,vol=vol)
                testrecord.append('1')
                response_check = False
                while response_check == False:
                    response = input("TEST: please indicate which note is higher pitched. (Reminder: type '1' or '2' to respond.)")
                    if (response == '1') | (response == '2'):
                        response_check = True
                        testresponses.append(response)
                    else:
                        print('Invalid response, please try again.')         
            else: #Low-high trial implemented
                play_sound(test_2,vol=vol)
                testrecord.append('2')
                response_check = False
                while response_check == False:
                    response = input("TEST: please indicate which note is higher pitched. (Reminder: type '1' or '2' to respond.)")
                    if (response == '1') | (response == '2'):
                        response_check = True
                        testresponses.append(response)
                    else:
                        print('Invalid response, please try again.')
        test_freq = test_freq+1

#CONFIRM TEST TRIAL RESPONSES - final 3 responses must be correct to proceed to experiment trials
if testrecord[2:5] == testresponses[2:5]:
    moveon_check = False
    while moveon_check == False:
        moveon = input("Thank you for completing the test prompts. To begin the study, please type 'y'.")
        if (moveon == 'y') | (moveon == 'Y'):
            moveon_check = True
        else:
            moveon_check = False
else:
    moveon_check = False
    stop_check = False
    while stop_check == False:
        stop = input ('Please review the instructions with the experimenter. Type "y" to pause the experiment.')
        if (stop == 'y') | (stop == 'Y'):
            stop_check = True
        else:
            stop_check = False

'''EXPERIMENT TRIALS''' 

#INITIALIZE EXPERIMENT
errors = 0 #Aggregates erroneous responses
base = 0 #Index into the base frequency
random.shuffle(f) #Randomizes the base frequencies for each subject

#FUNCTION TO RUN EXPERIMENT - currently set to draw from previously established parameters
def run(base=base,errors=errors,max_trials=max_trials,max_error=max_error,vol=vol,df=df,subjid=subjid):
    trialrecord = [] #Tracks whether the trial was high-low or low-high
    subjresponses = [] #Tracks subject responses
    dflist = [] #Tracks frequency differences
    baselist = [] #Tracks base frequencies
    subjidlist = [] #Tracks subject ID
    nlist = [] #Tracks trial number
    correct_update = 0 #Tracks consecutive correct responses
    error_update = 0 #Tracks erroneous responses
    n = 0
    trial_order = rng.integers(1,3,max_trials) #Randomizes high-low and low-high trials
    while (n < max_trials) & (errors < max_error):
        freq_a = base
        freq_b = freq_a+(freq_a*df) 
        note_A = make_tone(freq_a,duration)
        note_B = make_tone(freq_b,duration)
        test_A = np.concatenate((note_A,silence,note_B)) #Low-high trial; correct answer = 2
        test_B = np.concatenate((note_B,silence,note_A)) #High-low trial; correct answer = 1
        if trial_order[n] == 2: #Low-high trial implemented
            play_sound(test_A,vol=vol)
            trialrecord.append('2') 
            response_check = False
            while response_check == False:
                response = input("Please indicate which note is higher pitched.")
                if (response == '1') | (response == '2'):
                    response_check = True
                    subjresponses.append(response)
                else:
                    print('Invalid response, please try again.')
            if response == '2':
                correct_update = correct_update + 1 
            else:
                error_update = error_update + 1 
                errors = errors + 1 
                correct_update = 0 
        else: #High-low trial implemented
            play_sound(test_B,vol=vol)
            trialrecord.append('1')
            response_check = False
            while response_check == False:
                response = input("Please indicate which note is higher pitched.")
                if (response == '1') | (response == '2'):
                    response_check = True
                    subjresponses.append(response)
                else:
                    print('Invalid response, please try again.')
            if response == '1':
                correct_update = correct_update + 1
            else:
                error_update = error_update + 1
                errors = errors + 1
                correct_update = 0
        if (n == 0) & (error_update == 0): #Conditions for updating frequency difference
            dfl = 0.15
        if (n == 0) & (error_update == 1):
            dfl = 0.15
            error_update = 0
            df = df*np.sqrt(2)
        if (n != 0) & (correct_update == 2): 
            dfl = df 
            correct_update = 0
            df = df/np.sqrt(2)
        if (n != 0) & correct_update == 1:
            dfl = df
        if (n != 0) & error_update == 1: 
            dfl = df
            error_update = 0
            df = df*np.sqrt(2)
        n = n+1
        dflist.append(dfl)
        baselist.append(freq_a)
        subjidlist.append(subjid)
        nlist.append(n)
    return subjidlist, baselist, nlist, dflist, trialrecord, subjresponses 

#CONDUCT EXPERIMENT
datalist = []
if moveon_check == True:
    for base in f:
        data = run(base=base,errors=errors,max_trials=max_trials,max_error=max_error,vol=vol,df=df,subjid=subjid)
        datalist.append(data)

'''CONCLUSION OF EXPERIMENT'''

if moveon_check == True:
    con_check = False
    while con_check == False:
        con = input("Thank you for your participation! Please type 'y' to conclude the experiment.")
        if (con == 'y') | (con == 'Y'):
            con_check = True
        else:
            con_check = False

'''EXPORTING DATA'''

#ORGANIZE DATA
i = 0
all_base = []
all_df = []
all_correct = []
all_response = []
all_subjid = []
all_n = []

while i < len(datalist): 
    extract = datalist[i]
    all_subjid.append(extract[0]) #Extract list of subject IDs
    all_base.append(extract[1]) #Extract list of base frequencies
    all_n.append(extract[2]) #Extract list of trial numbers
    all_df.append(extract[3]) #Extract list of frequency differences
    all_correct.append(extract[4]) #Extract list of correct repsonses
    all_response.append(extract[5]) #Extract list of subject responses
    i = i+1

#FORMAT COLUMNS - flattens nested sublists in each extracted list 
col_subjid = [] 
for subj_sublist in all_subjid:
    for subj_item in subj_sublist:
        col_subjid.append(subj_item)

col_base = [] 
for base_sublist in all_base:
    for base_item in base_sublist:
        col_base.append(base_item)

col_n = [] 
for n_sublist in all_n:
    for n_item in n_sublist:
        col_n.append(n_item)

col_df = [] 
for df_sublist in all_df:
    for df_item in df_sublist:
        col_df.append(df_item)

j = 0 #Additionally converts frequency difference proportion into actual frequency difference in Hz
newcol = []
col_newdf = []
while j < len(col_df):
    if col_base[j] == 440:
        newdf = col_df[j]*440
    elif col_base[j] == 880:
        newdf = col_df[j]*880
    elif col_base[j] == 1760:
        newdf = col_df[j]*1760
    col_newdf.append(newdf)
    j = j+1

col_correct = [] 
for correct_sublist in all_correct:
    for correct_item in correct_sublist:
        col_correct.append(correct_item)

col_response = [] 
for response_sublist in all_response:
    for response_item in response_sublist:
        col_response.append(response_item)

#SAVE DATA AS .CSV - e.g. Experiment_Subject1.csv
dataframe = pd.DataFrame(columns = ['Subject','Base Frequency of Stimulus A','Trial Number','Frequency Difference (Hz)','Subject Response','Correct Response'])
dataframe['Subject'] = col_subjid
dataframe['Base Frequency of Stimulus A'] = col_base
dataframe['Trial Number'] = col_n
dataframe['Frequency Difference (Hz)'] = col_newdf
dataframe['Subject Response'] = col_response
dataframe['Correct Response'] = col_correct

filename = f"Experiment_Subject{subjid}.csv"
dataframe.to_csv(filename,index = False)